In [ ]:
!unzip /content/test.zip

In [2]:
from itertools import combinations_with_replacement
import itertools
import numpy as np
from skimage import filters, feature
from skimage import img_as_float32
from joblib import Parallel, delayed

try:
    from sklearn.exceptions import NotFittedError

    has_sklearn = True
except ImportError:
    has_sklearn = False

    class NotFittedError(Exception):
        pass


def _texture_filter(gaussian_filtered):
    H_elems = [
        np.gradient(np.gradient(gaussian_filtered)[ax0], axis=ax1)
        for ax0, ax1 in combinations_with_replacement(range(gaussian_filtered.ndim), 2)
    ]
    eigvals = feature.hessian_matrix_eigvals(H_elems)
    return eigvals


def _mutiscale_basic_features_singlechannel(
    img, intensity=True, edges=True, texture=True, sigma_min=0.5, sigma_max=16
):
    """Features for a single channel nd image.

    Parameters
    ----------
    """
    # computations are faster as float32
    img = np.ascontiguousarray(img_as_float32(img))
    sigmas = np.logspace(
        np.log2(sigma_min),
        np.log2(sigma_max),
        num=int(np.log2(sigma_max) - np.log2(sigma_min) + 1),
        base=2,
        endpoint=True,
    )
    all_filtered = Parallel(n_jobs=-1, prefer="threads")(
        delayed(filters.gaussian)(img, sigma) for sigma in sigmas
    )
    features = []
    if intensity:
        features += all_filtered
    if edges:
        all_edges = Parallel(n_jobs=-1, prefer="threads")(
            delayed(filters.sobel)(filtered_img) for filtered_img in all_filtered
        )
        features += all_edges
    if texture:
        all_texture = Parallel(n_jobs=-1, prefer="threads")(
            delayed(_texture_filter)(filtered_img) for filtered_img in all_filtered
        )
        features += itertools.chain.from_iterable(all_texture)
    return features


def multiscale_basic_features(
    image,
    multichannel=True,
    intensity=True,
    edges=True,
    texture=True,
    sigma_min=0.5,
    sigma_max=16,
):
    """Local features for a single- or multi-channel nd image.

    Intensity, gradient intensity and local structure are computed at
    different scales thanks to Gaussian blurring.

    Parameters
    ----------
    image : ndarray
        Input image, which can be grayscale or multichannel.
    multichannel : bool, default False
        True if the last dimension corresponds to color channels.
    intensity : bool, default True
        If True, pixel intensities averaged over the different scales
        are added to the feature set.
    edges : bool, default True
        If True, intensities of local gradients averaged over the different
        scales are added to the feature set.
    texture : bool, default True
        If True, eigenvalues of the Hessian matrix after Gaussian blurring
        at different scales are added to the feature set.
    sigma_min : float, optional
        Smallest value of the Gaussian kernel used to average local
        neighbourhoods before extracting features.
    sigma_max : float, optional
        Largest value of the Gaussian kernel used to average local
        neighbourhoods before extracting features.

    Returns
    -------
    features : np.ndarray
        Array of shape ``(n_features,) + image.shape``
    """
    if image.ndim >= 3 and multichannel:
        all_results = (
            _mutiscale_basic_features_singlechannel(
                image[..., dim],
                intensity=intensity,
                edges=edges,
                texture=texture,
                sigma_min=sigma_min,
                sigma_max=sigma_max,
            )
            for dim in range(image.shape[-1])
        )
        features = list(itertools.chain.from_iterable(all_results))
    else:
        features = _mutiscale_basic_features_singlechannel(
            image,
            intensity=intensity,
            edges=edges,
            texture=texture,
            sigma_min=sigma_min,
            sigma_max=sigma_max,
        )
    return np.array(features, dtype=np.float32)


def fit_segmenter(labels, features, clf):
    """
    Segmentation using labeled parts of the image and a classifier.

    Parameters
    ----------
    labels : ndarray of ints
        Image of labels. Labels >= 1 correspond to the training set and
        label 0 to unlabeled pixels to be segmented.
    features : ndarray
        Array of features, with the first dimension corresponding to the number
        of features, and the other dimensions correspond to ``labels.shape``.
    clf : classifier object
        classifier object, exposing a ``fit`` and a ``predict`` method as in
        scikit-learn's API, for example an instance of
        ``RandomForestClassifier`` or ``LogisticRegression`` classifier.

    Returns
    -------
    output : ndarray
        Labeled array, built from the prediction of the classifier trained on
        ``labels``.
    clf : classifier object
        classifier trained on ``labels``

    Raises
    ------
    NotFittedError if ``self.clf`` has not been fitted yet (use ``self.fit``).
    """
    training_data = features[:, labels > 0].T
    training_labels = labels[labels > 0].ravel()
    clf.fit(training_data, training_labels)
    data = features[:, labels == 0].T
    predicted_labels = clf.predict(data)
    output = np.copy(labels)
    output[labels == 0] = predicted_labels
    return output, clf


def predict_segmenter(features, clf):
    """
    Segmentation of images using a pretrained classifier.

    Parameters
    ----------
    features : ndarray
        Array of features, with the first dimension corresponding to the number
        of features, and the other dimensions are compatible with the shape of
        the image to segment.
    clf : classifier object
        trained classifier object, exposing a ``predict`` method as in
        scikit-learn's API, for example an instance of
        ``RandomForestClassifier`` or ``LogisticRegression`` classifier. The
        classifier must be already trained, for example with
        :func:`fit_segmenter`.
    features_func : function, optional
        function computing features on all pixels of the image, to be passed
        to the classifier. The output should be of shape
        ``(m_features, *labels.shape)``. If None,
        :func:`multiscale_basic_features` is used.

    Returns
    -------
    output : ndarray
        Labeled array, built from the prediction of the classifier.
    """
    sh = features.shape
    features = features.reshape((sh[0], np.prod(sh[1:]))).T
    try:
        predicted_labels = clf.predict(features)
    except NotFittedError:
        raise NotFittedError(
            "You must train the classifier `clf` first"
            "for example with the `fit_segmenter` function."
        )
    output = predicted_labels.reshape(sh[1:])
    return output

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from glob import glob
from PIL import Image

In [3]:
clf = RandomForestClassifier(max_depth = 5)

In [4]:
clf = DecisionTreeClassifier(max_depth = 5)

In [5]:
label_paths = sorted(list(glob('/content/ds/labels/*')))
img_paths = sorted(list(glob('/content/ds/images/*')))

In [6]:
len(img_paths)

50

In [7]:
len(label_paths)

50

In [8]:
Shape = ()
label_Shape = ()

In [9]:
labels = []
images = []
for i in range(5):
  label = np.array(Image.open(label_paths[i]))
  img = np.array(Image.open(img_paths[i]))
  Shape = img.shape
  label_Shape = label.shape
  labels.append(label)
  images.append(img)
  print(i)
labels = np.array(labels)
images = np.array(images)

0
1
2
3
4


In [ ]:
images

In [ ]:
multiscale_basic_features(images)

In [11]:
output, clf = fit_segmenter(labels, multiscale_basic_features(images), clf)

In [47]:
# !pip install numpy==1.21.6

In [ ]:
labels = []
images = []
for i in range(5, 50):
  label = np.array(Image.open(label_paths[i]))
  img = np.array(Image.open(img_paths[i]))
  print(img.shape)
  print(label.shape)
  if img.shape == Shape and label.shape == label_Shape:
    labels.append(label)
    images.append(img)
labels = labels[:10]
images = images[:10]
labels = np.array(labels)
images = np.array(images)

In [ ]:
# multiscale_basic_features(images)

In [15]:
def IoU_Dice(pred, targ):
  overlap = pred * targ # Logical AND
  union = pred + targ - overlap # Logical OR
  IOU = overlap.sum() / float(union.sum())
  DICE = overlap.sum() * 2.0 / (pred.sum() + targ.sum())
  return IOU, DICE

In [13]:
predicted_labels = predict_segmenter(multiscale_basic_features(images), clf)

In [16]:
IOU_av = 0.
DICE_av = 0.
for i in range(10):
  IOU, DICE = IoU_Dice(predicted_labels[i], labels[i])
  IOU_av += IOU
  DICE_av += DICE
IOU_av /= 10
DICE_av /= 10

In [17]:
IOU_av

0.0020594646780074384

In [18]:
DICE_av

0.0026405218709026398